# CSE547 - Colab 2
## Frequent Pattern Mining in Spark

Adapted from Stanford CS246

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u242-b08-0ubuntu3~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
id='1Pu5gi1gb4jRD22_njTy2u77CVGsVuVQe'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('browsing.txt')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import functions as F

Let's initialize the Spark context.

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050").set("spark.executor.memory", "8g")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [8]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-04-13 21:56:29--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.73.195.55, 52.70.214.4, 52.4.202.19, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.73.195.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.13M  6.61MB/s    in 2.0s    

2020-04-13 21:56:32 (6.61 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://b2fd0832.ngrok.io


### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [168]:
browsing = sc.textFile('browsing.txt')

data = browsing\
  .map(lambda x:[[item.strip() for item in x.strip().split(' ') if item]])\
  .toDF().toPandas()

num_rows = data.count()
s = 100/int(num_rows)

print(data)

#lol=[]
#for i in sequence(0,num_rows-1):
#  lol.append(data.toDF().iloc(i))

#print(lol[2])

                                                      _1
0      [FRO11987, ELE17451, ELE89019, SNA90258, GRO99...
1      [GRO99222, GRO12298, FRO12685, ELE91550, SNA11...
2      [ELE17451, GRO73461, DAI22896, SNA99873, FRO86...
3      [ELE17451, ELE37798, FRO86643, GRO56989, ELE23...
4      [ELE17451, SNA69641, FRO86643, FRO78087, SNA11...
...                                                  ...
31096  [SNA30755, ELE97341, DAI42493, ELE74009, DAI22...
31097  [DAI87693, DAI35347, FRO31317, GRO21037, GRO88...
31098  [FRO53271, ELE53126, FRO31317, DAI83948, GRO88...
31099  [FRO41319, ELE14480, SNA62128, ELE92920, FRO99...
31100  [DAI45339, SNA59903, DAI62779, DAI92600, DAI42...

[31101 rows x 1 columns]


In [169]:
lol = [l for l in data['_1']]
te = TransactionEncoder()
te_ary = te.fit(lol).transform(lol)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,DAI11153,DAI11223,DAI11238,DAI11257,DAI11261,DAI11273,DAI11290,DAI11299,DAI11375,DAI11462,DAI11541,DAI11552,DAI11555,DAI11582,DAI11613,DAI11695,DAI11707,DAI11778,DAI11927,DAI11946,DAI11995,DAI12036,DAI12139,DAI12152,DAI12275,DAI12437,DAI12448,DAI12460,DAI12521,DAI12535,DAI12558,DAI12566,DAI12680,DAI12683,DAI12698,DAI12720,DAI12740,DAI12770,DAI12821,DAI12844,...,SNA98597,SNA98598,SNA98604,SNA98612,SNA98629,SNA98720,SNA98960,SNA99004,SNA99039,SNA99143,SNA99173,SNA99182,SNA99196,SNA99310,SNA99326,SNA99341,SNA99386,SNA99393,SNA99404,SNA99413,SNA99492,SNA99499,SNA99515,SNA99618,SNA99634,SNA99635,SNA99654,SNA99690,SNA99752,SNA99767,SNA99791,SNA99814,SNA99861,SNA99870,SNA99873,SNA99886,SNA99895,SNA99918,SNA99924,SNA99941
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31096,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
31097,False,False,False,Fals

In [147]:
df[df['DAI11153']]

,DAI11153,DAI11223,DAI11238,DAI11257,DAI11261,DAI11273,DAI11290,DAI11299,DAI11375,DAI11462,DAI11541,DAI11552,DAI11555,DAI11582,DAI11613,DAI11695,DAI11707,DAI11778,DAI11927,DAI11946,DAI11995,DAI12036,DAI12139,DAI12152,DAI12275,DAI12437,DAI12448,DAI12460,DAI12521,DAI12535,DAI12558,DAI12566,DAI12680,DAI12683,DAI12698,DAI12720,DAI12740,DAI12770,DAI12821,DAI12844,...,SNA98597,SNA98598,SNA98604,SNA98612,SNA98629,SNA98720,SNA98960,SNA99004,SNA99039,SNA99143,SNA99173,SNA99182,SNA99196,SNA99310,SNA99326,SNA99341,SNA99386,SNA99393,SNA99404,SNA99413,SNA99492,SNA99499,SNA99515,SNA99618,SNA99634,SNA99635,SNA99654,SNA99690,SNA99752,SNA99767,SNA99791,SNA99814,SNA99861,SNA99870,SNA99873,SNA99886,SNA99895,SNA99918,SNA99924,SNA99941
13917,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
14585,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
17989,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
18020,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
18108,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
18249,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
21404,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,Fal

,DAI11153,DAI11223,DAI11238,DAI11257,DAI11261,DAI11273,DAI11290,DAI11299,DAI11375,DAI11462,DAI11541,DAI11552,DAI11555,DAI11582,DAI11613,DAI11695,DAI11707,DAI11778,DAI11927,DAI11946,DAI11995,DAI12036,DAI12139,DAI12152,DAI12275,DAI12437,DAI12448,DAI12460,DAI12521,DAI12535,DAI12558,DAI12566,DAI12680,DAI12683,DAI12698,DAI12720,DAI12740,DAI12770,DAI12821,DAI12844,...,SNA98597,SNA98598,SNA98604,SNA98612,SNA98629,SNA98720,SNA98960,SNA99004,SNA99039,SNA99143,SNA99173,SNA99182,SNA99196,SNA99310,SNA99326,SNA99341,SNA99386,SNA99393,SNA99404,SNA99413,SNA99492,SNA99499,SNA99515,SNA99618,SNA99634,SNA99635,SNA99654,SNA99690,SNA99752,SNA99767,SNA99791,SNA99814,SNA99861,SNA99870,SNA99873,SNA99886,SNA99895,SNA99918,SNA99924,SNA99941
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31096,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
31097,False,False,False,Fals

In [175]:
frq_items  = apriori(df, min_support=s, use_colnames=True,max_len=3)

# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, 
                          metric='support',
                          min_threshold = s) 
rules = rules.sort_values(desc('support')) 
rules.head(1)

TypeError: ignored

Compute how many Frequent Itemsets and Association Rules were generated by running FP-growth.


In [170]:
association_rules.
[length(association_rules.itemsets)>1]

# Count frequent itemsets.
#print(model.freqItemsets.count())

# Count generated association rules.
#print(model.associationRules.count())

TypeError: ignored

Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many Frequent Itemsets and Association Rules were generated.

In [123]:
fpGrowth = FPGrowth(itemsCol='products', minSupport=s, minConfidence=0.5)
model2 = fpGrowth.fit(joined)

# Count frequent itemsets.
print(model2.freqItemsets.count())

# Count generated association rules.
print(model2.associationRules.count())

NameError: ignored

Now, for 
```minsupport=0.001```, 
visualize the top Frequent Itemsets and Association Rules.

In addition for
```minsupport=0.001```, 
calculate the "interest" of the Association Rules. 

(Hint: One way to do this by joining the Frequent Itemsets with the Association Rules)




In [0]:
# Count frequent itemsets.
model2.freqItemsets.sort(desc('freq')).show()

# Count generated association rules.
model2.associationRules.show()

+--------------------+-----+
|               items| freq|
+--------------------+-----+
|            [Banana]|18726|
|[Bag of Organic B...|15480|
|[Organic Strawber...|10894|
|[Organic Baby Spi...| 9784|
|       [Large Lemon]| 8135|
|   [Organic Avocado]| 7409|
|[Organic Hass Avo...| 7293|
|      [Strawberries]| 6494|
|             [Limes]| 6033|
|[Organic Raspberr...| 5546|
|[Organic Blueberr...| 4966|
|[Organic Whole Milk]| 4908|
|  [Organic Cucumber]| 4613|
|  [Organic Zucchini]| 4589|
|[Organic Yellow O...| 4290|
|    [Organic Garlic]| 4158|
|[Seedless Red Gra...| 4059|
|         [Asparagus]| 3868|
|[Organic Grape To...| 3823|
| [Organic Red Onion]| 3818|
+--------------------+-----+
only showing top 20 rows

+--------------------+--------------------+------------------+------------------+
|          antecedent|          consequent|        confidence|              lift|
+--------------------+--------------------+------------------+------------------+
|[Organic Kiwi, Or...|[Bag of Or

In [131]:
int(data.count())

31101

In [0]:
num_baskets = joined.count()

joined_model = model2.freqItemsets.\
  join(model2.associationRules,
    model2.freqItemsets.items==model2.associationRules.consequent 
).withColumn('interest',abs(col('confidence') - col('freq')/num_baskets ))\
.sort(desc('interest'))
  
joined_model.toPandas().show()

                       items   freq  ...      lift  interest
0   [Bag of Organic Bananas]  15480  ...  5.072272  0.480445
1   [Bag of Organic Bananas]  15480  ...  4.635331  0.428895
2   [Bag of Organic Bananas]  15480  ...  4.627719  0.427997
3   [Bag of Organic Bananas]  15480  ...  4.587387  0.423239
4   [Bag of Organic Bananas]  15480  ...  4.504745  0.413489
5   [Bag of Organic Bananas]  15480  ...  4.477905  0.410322
6   [Bag of Organic Bananas]  15480  ...  4.416854  0.403119
7   [Bag of Organic Bananas]  15480  ...  4.382495  0.399066
8   [Bag of Organic Bananas]  15480  ...  4.357585  0.396127
9                   [Banana]  18726  ...  3.753633  0.392995
10  [Bag of Organic Bananas]  15480  ...  4.278898  0.386843

[11 rows x 7 columns]


In [0]:
a= joined_model.toPandas()
print(a.head(5))
print([a for a in a.head(1)['antecedent']])

                      items   freq  ...      lift  interest
0  [Bag of Organic Bananas]  15480  ...  5.072272  0.480445
1  [Bag of Organic Bananas]  15480  ...  4.635331  0.428895
2  [Bag of Organic Bananas]  15480  ...  4.627719  0.427997
3  [Bag of Organic Bananas]  15480  ...  4.587387  0.423239
4  [Bag of Organic Bananas]  15480  ...  4.504745  0.413489

[5 rows x 7 columns]
[['Organic Raspberries', 'Organic Hass Avocado', 'Organic Strawberries']]


To conclude, go to Gradescope and read the remaining questions. We will ask you to inspect the resulting dataframes, and report a few results.

Once you obtained the desired results, **submit your solution for this Colab**!